## Setup

In [3]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2015, day=14)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [4]:
# Import performance checking utility
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import check_example, time_solution

## Part a

In [8]:
# Functions
def solve_a(input_data: str, *, total_time: int = 2503) -> int:
    """Find the location of the fastest reindeer after a given time."""
    distances: list[int] = []

    for speed, fly_time, rest_time in [
        [*map(int, (li[3], li[6], li[-2]))] for li in (line.split() for line in input_data.splitlines())
    ]:
        # Calculate distance traveled
        cycle_time = fly_time + rest_time
        full_cycles, remaining_time = divmod(total_time, cycle_time)
        distances.append(speed * (full_cycles * fly_time + min(remaining_time, fly_time)))

    return max(distances)

In [12]:
# Correctness check
check_example(solve_a, example, "a", total_time=1000)

solve_a found answer 1120, which is the correct solution for part A!


True

In [31]:
# Performance check
time_a = time_solution(solve_a, input_data, time_unit="us")

solve_a takes 13.04 μs


In [15]:
# Submit answer
puzzle.answer_a = solve_a(input_data)

That's the right answer!  You are one gold star closer to powering the weather machine. [Continue to Part Two]


## Part b

### Iterative approach
Instead of tracking distances at some time T, we now need to compute the distances at every time step up to T, and award points based on who is in the lead at each time step. Let's start with a simple iterative approach.

In [45]:
# Functions
def solve_b_iterative(input_data: str, *, total_time: int = 2503) -> int:
    """Find winning reindeer based on points after a given time."""
    reindeer_info = [[*map(int, (li[3], li[6], li[-2]))] for li in (line.split() for line in input_data.splitlines())]
    distances_at_t: list[list[int]] = []
    for speed, fly_time, rest_time in reindeer_info:
        cycle_time = fly_time + rest_time

        # Construct the location matrix
        reindeer_location = []
        pos = 0
        for t in range(total_time):
            if t % cycle_time < fly_time:
                # Reindeer is flying
                pos += speed
            reindeer_location.append(pos)

        distances_at_t.append(reindeer_location)

    points = [0] * len(reindeer_info)
    for t in range(total_time):
        max_dist_at_t = max(loc[t] for loc in distances_at_t)
        for reindeer, dist in enumerate(distances_at_t):
            if dist[t] == max_dist_at_t:
                points[reindeer] += 1  # Award a point

    return max(points)

In [46]:
# Performance check
time_b_iterative = time_solution(solve_b_iterative, input_data)

solve_b_iterative takes 2.43 ms


### Vectorized approach
Let's see if we can speed things up with a vectorized approach using NumPy.

In [47]:
# Imports
import numpy as np

In [60]:
# Functions
def solve_b_numpy(input_data: str, *, total_time: int = 2503) -> int:
    """Find the winning reindeer based on points after a given time using a vectorized approach."""
    reindeer_info = np.array(
        [[int(r[3]), int(r[6]), int(r[-2])] for r in [line.split() for line in input_data.splitlines()]], dtype=np.int16
    )

    # Unpack info of n reindeer into three (n,1) vectors
    speeds = reindeer_info[:, 0:1]
    fly_time = reindeer_info[:, 1:2]
    cycle_time = fly_time + reindeer_info[:, 2:3]

    # (T,) array of time steps
    t = np.arange(total_time, dtype=np.int64)

    # (n, T) mask of whether each reindeer is flying at each second
    is_flying = np.mod(t, cycle_time) < fly_time

    # (n, T) positions after each second
    positions = np.cumsum(is_flying * speeds, axis=1)

    # (n, T) leader mask per second
    leaders = positions == positions.max(axis=0)

    # We can simply sum the boolean leader mask to get points per reindeer, as True is counted as 1
    points = leaders.sum(axis=1)

    return int(points.max())

In [62]:
# Performance check
time_b_numpy = time_solution(solve_b_numpy, input_data)
print(f"This is {time_b_iterative / time_b_numpy:.1f}x faster than the iterative approach.")

solve_b_numpy takes 0.15 ms
This is 16.0x faster than the iterative approach.


In [ ]:
# Submit answer
puzzle.answer_b = solve_b_numpy(input_data)

That's the right answer!  You are one gold star closer to powering the weather machine.You have completed Day 14! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
